Cleanser for the data in the elections demo.  Takes in elections.csv, a spreadsheet with states in the rows and years in the columns.  A group of columns represents a year, and a column the votes for a candidate in the year.  There is a total for each year as well, so the minimum is 3: (republicam, democrat, total) First row is year numbers.  Second is candidate name and party, separated by a dash.  3-54 is votes.  First column is states, and first state is Nationwide.  We will output a flat table, with columns (Year, Candidate, Party, State, Vote, Pct).
Some of this file is one-off cleansing, the rest is the start of the spreadsheet-ingress widget.
The Notebook is a single cell that reads elections.csv and writes election-rows.csv.  Written  as a Notebook for convenience

In [ ]:
#
# Open the file and read the rows.  There are few enough that we just read them
#
f = open('elections.csv', 'r')
election_reader = csv.reader(f)
rows = [row for row in election_reader]
f.close()
# 
# Fill in the year for each column.  The first column of a year group has a year in it, the remainder are blank.  
# Substitute a blank for the last year seen.  The result is in real_years
#
years = rows[0]
last = None
real_years = []
for year in years:
    if (len(year) == 0):
        if (last): real_years.append(last)
        else: real_years.append(year)
    else:
        real_years.append(year)
        last = year
#
# Split the fields (candidate-party), e.g. "Clinton, Hillary - Democratic" into two fields "Clinton, Hillary" and "Democratic".  The 
# first field is in candidates, the second is in parties
#
candidate_row = rows[1]
candidate_fields = [field.split(' - ') for field in candidate_row]
candidates = [field[0] for field in candidate_fields]
parties = [field[1] if len(field) == 2 else field[0] for field in candidate_fields]
#
# Housekeeping.  Convert the values into integers in each row.  Pandas would do this for us, but we have to do it ourselves.  Mostly,
# this just means substituting 0's for ints.  Results are in state_rows
#
vals = rows[2:]
state_rows = []
for row in vals:
    values = [0 if val == '' else int(val) for val in row[1:]]
    state_rows.append([row[0]] + values)
#
# We want to append the percentage of the vote to each candidate.  This just involves dividing by the total for the
# state and year.  Just get the index for the total fields per year (they are the same for each year).  The totals list
# will be the index of the greatest PRECEDING column whose party was 'Total'.  So, for example, if parties[3] = 'Total'
# and parties[2] = 'Total', totals[2] = totals[3] = 2
#

current_total = 0
totals = []
for i in range(len(real_years)):
    if parties[i] == 'Total': current_total = i
    totals.append(current_total)

#
# set up the header for the output rows
#

output_rows = [['Year', 'Candidate','Party', 'State', 'Votes', 'Pct']]

#
# Compute and write out the records; onlyu records with nonzero votes are included.  Pct is computed 
# inline for each record, using the totals list just computed
#
for state_row in state_rows:
    state = state_row[0]
    for i in range(1, len(years)):
        votes = state_row[i]
        if votes == '': continue
        total = state_row[totals[i]]
        pct = state_row[i] * 100.0/total if total > 0 else 0
        output_rows.append([real_years[i], candidates[i], parties[i], state, state_row[i], pct])

#
# Write out the table to election-rows.csv
#
f = open('election-rows.csv', 'w')
w = csv.writer(f)
w.writerows(output_rows)
f.close()
'%d records written to election-rows.csv' % len(output_rows)